In [4]:
!cp /content/drive/My\ Drive/m5/m5-forecasting-accuracy.zip /content/
!unzip /content/m5-forecasting-accuracy.zip 

Archive:  /content/m5-forecasting-accuracy.zip
  inflating: calendar.csv            
  inflating: sales_train_evaluation.csv  
  inflating: sales_train_validation.csv  
  inflating: sample_submission.csv   
  inflating: sell_prices.csv         


In [5]:
import pandas as pd
import numpy as np

In [6]:
timesteps = 28
start_day = 350

In [7]:
df = pd.read_csv('sales_train_evaluation.csv')
df.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,3,0,1,1,1,3,0,1,1,0,0,0,2,0,3,5,0,0,1,1,0,2,1,2,2,1,0,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,2,2,1,2,1,1,1,0,1,1,1,0,0,1,1,0,2,1,0,0,0,0,2,1,3,0,0,1,0,1,0,2,0,0,0,2,3,0,1


In [8]:
#To reduce memory usage
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

In [9]:
df = downcast_dtypes(df)

In [10]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: int16(1941), object(6)
memory usage: 114.3+ MB


In [11]:
 df = df.T[5 + start_day:]

In [12]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,30450,30451,30452,30453,30454,30455,30456,30457,30458,30459,30460,30461,30462,30463,30464,30465,30466,30467,30468,30469,30470,30471,30472,30473,30474,30475,30476,30477,30478,30479,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
d_350,0,0,0,2,0,0,0,0,2,0,0,0,0,2,4,0,2,0,0,1,0,2,0,0,2,0,0,3,0,0,1,14,0,0,0,0,0,1,0,0,...,0,0,0,0,14,1,1,0,0,0,0,0,19,0,2,0,11,8,0,4,2,0,0,0,0,0,5,0,0,0,0,7,1,0,6,0,2,1,0,0
d_351,0,0,0,2,0,0,0,24,3,2,0,1,0,4,17,0,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,16,0,0,0,0,0,2,0,0,...,0,0,0,0,14,0,1,0,0,1,0,1,60,4,2,0,10,2,0,9,2,9,0,0,7,0,6,0,0,0,0,9,1,0,11,0,0,1,0,0
d_352,0,0,0,0,0,0,0,9,0,2,0,0,0,2,0,0,3,0,0,0,0,1,2,0,0,0,0,0,0,0,0,9,0,0,0,3,0,1,0,0,...,0,0,0,0,18,5,0,0,0,1,0,0,30,4,0,0,6,5,0,8,7,12,0,0,0,0,3,0,0,1,0,5,4,0,8,0,1,2,0,0
d_353,0,0,0,4,2,0,0,2,1,1,0,1,0,0,10,0,0,0,0,0,0,1,2,0,0,0,0,3,0,0,0,3,0,0,0,0,0,1,0,0,...,0,0,0,0,18,5,2,0,0,1,0,0,11,1,0,0,10,4,0,11,9,7,0,0,1,0,16,0,0,2,0,15,2,0,3,0,1,2,0,0
d_354,0,1,0,2,0,0,0,7,1,0,0,0,0,1,2,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,21,0,1,0,0,0,1,0,0,...,0,0,0,0,13,1,1,0,0,1,0,0,15,7,0,0,4,1,0,3,2,2,0,0,1,0,9,0,0,0,0,5,1,0,3,0,0,0,0,0


In [13]:
calendar = pd.read_csv("calendar.csv")


In [14]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [15]:
#Create dataframe with zeros for 1969 days in the calendar
days_before_event = pd.DataFrame(np.zeros((1969,1)))

In [16]:
for x,y in calendar.iterrows():
    if((pd.isnull(calendar["event_name_1"][x])) == False):
           days_before_event[0][x-1] = 1 

In [17]:
days_before_event

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
1964,0.0
1965,0.0
1966,0.0
1967,1.0


In [18]:
del calendar

In [19]:
#"daysBeforeEventTest" will be used as input for predicting (We will forecast the days 1913-1941)
days_before_event_test = days_before_event[1941:1969]
#"daysBeforeEvent" will be used for training as a feature.
days_before_event = days_before_event[start_day:1942]

In [20]:
days_before_event_test.head()

,0
1941,0.0
1942,0.0
1943,0.0
1944,0.0
1945,0.0


In [21]:
#Before concatanation with our main data "dt", indexes are made same and column name is changed to "oneDayBeforeEvent"
days_before_event.columns = ["one_day_before_event"]
days_before_event.index = df.index

In [22]:
days_before_event

,one_day_before_event
d_350,0.0
d_351,1.0
d_352,0.0
d_353,0.0
d_354,0.0
...,...
d_1937,0.0
d_1938,0.0
d_1939,0.0
d_1940,0.0


In [23]:
df = pd.concat([df, days_before_event], axis = 1)

df.columns

Index([                     0,                      1,                      2,
                            3,                      4,                      5,
                            6,                      7,                      8,
                            9,
       ...
                        30481,                  30482,                  30483,
                        30484,                  30485,                  30486,
                        30487,                  30488,                  30489,
       'one_day_before_event'],
      dtype='object', length=30491)

In [24]:
#Feature Scaling
#Scale the features using min-max scaler in range 0-1
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
df_scaled = sc.fit_transform(df)

In [25]:
df_scaled

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.6       , 0.        , 0.5       , ..., 0.08333333, 0.16666667,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.08333333, 0.41666667,
        0.        ],
       [0.2       , 0.        , 0.16666667, ..., 0.        , 0.08333333,
        0.        ]])

In [26]:
X_train = []
y_train = []
for i in range(timesteps, 1941 - start_day):
    X_train.append(df_scaled[i-timesteps:i])
    y_train.append(df_scaled[i][0:30490]) 
    #İmportant!! if extra features are added (like oneDayBeforeEvent) 
    #use only sales values for predictions (we only predict sales) 
    #this is why 0:30490 columns are choosen

In [27]:
del df_scaled


In [28]:
#Convert to np array to be able to feed the LSTM model
X_train = np.array(X_train)
y_train = np.array(y_train)
print(X_train.shape)
print(y_train.shape)

(1563, 28, 30491)
(1563, 30490)


In [29]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
layer_1_units=50
regressor.add(LSTM(units = layer_1_units, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
layer_2_units=400
regressor.add(LSTM(units = layer_2_units, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
layer_3_units=400
regressor.add(LSTM(units = layer_3_units))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 30490))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
epoch_no=32
batch_size_RNN=44
regressor.fit(X_train, y_train, epochs = epoch_no, batch_size = batch_size_RNN)

Using TensorFlow backend.


Epoch 1/32
1563/1563 [==============================] - 12s 7ms/step - loss: 0.0180
Epoch 2/32
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0157
Epoch 3/32
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0152
Epoch 4/32
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0149
Epoch 5/32
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0148
Epoch 6/32
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0146
Epoch 7/32
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0144
Epoch 8/32
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0142
Epoch 9/32
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0141
Epoch 10/32
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0140
Epoch 11/32
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0139
Epoch 12/32
1563/1563 [==============================] - 9s 6ms/step - loss: 0.0138


In [30]:
inputs= df[-timesteps:]
inputs = sc.transform(inputs)

In [31]:
X_test = []
X_test.append(inputs[0:timesteps])
X_test = np.array(X_test)
predictions = []

for j in range(timesteps,timesteps + 28):
#X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    predicted_stock_price = regressor.predict(X_test[0,j - timesteps:j].reshape(1, timesteps, 30491))
    testInput = np.column_stack((np.array(predicted_stock_price), days_before_event_test[0][1941 + j - timesteps]))
    X_test = np.append(X_test, testInput).reshape(1,j + 1,30491)
    predicted_stock_price = sc.inverse_transform(testInput)[:,0:30490]
    predictions.append(predicted_stock_price)

In [32]:
import time

submission = pd.DataFrame(data=np.array(predictions).reshape(28,30490))

submission = submission.T
    
submission = pd.concat((submission, submission), ignore_index=True)

sample_submission = pd.read_csv("sample_submission.csv")
    
idColumn = sample_submission[["id"]]
    
submission[["id"]] = idColumn  

cols = list(submission.columns)
cols = cols[-1:] + cols[:-1]
submission = submission[cols]

colsdeneme = ["id"] + [f"F{i}" for i in range (1,29)]

submission.columns = colsdeneme

currentDateTime = time.strftime("%d%m%Y_%H%M%S")

submission.to_csv("submission.csv", index=False)

In [33]:
submission.iloc[30489]


id     FOODS_3_827_WI_3_validation
F1                         1.58831
F2                         1.28973
F3                         1.02057
F4                        0.856718
F5                         1.01328
F6                         1.26358
F7                         1.13287
F8                         1.48476
F9                         1.23234
F10                       0.962977
F11                       0.815992
F12                       0.975849
F13                        1.19704
F14                       0.980221
F15                        1.29415
F16                        1.07563
F17                        0.85187
F18                       0.735713
F19                       0.896311
F20                        1.12939
F21                       0.929045
F22                        1.20353
F23                       0.987114
F24                        0.80299
F25                       0.690406
F26                       0.846421
F27                        1.12684
F28                 

In [34]:
submission.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.815728,0.867909,0.885613,0.913348,1.119897,1.573209,1.274745,0.857075,0.890398,0.896196,0.913773,1.101511,1.527933,1.307716,0.871239,0.905684,0.932150,0.969735,1.181465,1.659744,1.346130,0.858475,0.918069,0.968440,1.026663,1.271037,1.764581,1.331670
1,HOBBIES_1_002_CA_1_validation,0.336695,0.317910,0.272589,0.218279,0.195985,0.286284,0.306071,0.270129,0.266546,0.232413,0.198262,0.196822,0.286122,0.256412,0.203733,0.212345,0.197397,0.186676,0.201641,0.307134,0.247631,0.207471,0.217794,0.206440,0.188198,0.184374,0.310122,0.312195
2,HOBBIES_1_003_CA_1_validation,0.448287,0.489624,0.559372,0.650399,0.874187,1.235746,0.961267,0.408284,0.428914,0.499941,0.601798,0.847301,1.216873,0.974233,0.390108,0.391680,0.469961,0.581590,0.831690,1.206184,0.970285,0.425329,0.436554,0.515422,0.627798,0.884404,1.275125,1.009634
3,HOBBIES_1_004_CA_1_validation,1.329013,0.995822,1.013011,1.152658,1.383301,2.380000,3.756322,1.435231,0.978195,1.002211,1.193438,1.450916,2.412335,3.764455,1.545219,1.050833,1.090889,1.249738,1.349030,2.343520,3.756135,1.530640,1.134698,1.147326,1.215640,1.227924,2.339308,3.715334
4,HOBBIES_1_005_CA_1_validation,1.292862,1.352092,1.370208,1.319341,1.288823,1.509008,1.731043,1.204798,1.266085,1.272977,1.238550,1.233761,1.453690,1.686985,1.138728,1.180998,1.219308,1.238316,1.228266,1.488785,1.709804,1.180749,1.225334,1.299477,1.332768,1.293164,1.587970,1.787834
